In [ ]:
!pip install neowise

In [ ]:
import neowise as nw
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
from tensorflow.keras.datasets import mnist
from keras.utils import to_categorical

In [ ]:
# Data Preparation --- MNIST
(X_train_orig, Y_train_orig), (X_test_orig, Y_test_orig) = mnist.load_data()

# Preparing the data
Y_tr_resh = Y_train_orig.reshape(60000, 1)
Y_te_resh = Y_test_orig.reshape(10000, 1)
Y_tr_T = to_categorical(Y_tr_resh, num_classes=10)
Y_te_T = to_categorical(Y_te_resh, num_classes=10)
y_tr = Y_tr_T.T
y_te = Y_te_T.T


# Flattening of inputs
X_train_flatten = X_train_orig.reshape(X_train_orig.shape[0], -1).T
X_test_flatten = X_test_orig.reshape(X_test_orig.shape[0], -1).T

# Preprocessing of Inputs
X_tr = X_train_flatten.T / 255.
X_te = X_test_flatten.T / 255.
num_classes = y_tr.shape[0]
m_tr = X_tr.shape[0]
m_te = X_te.shape[0]

In [ ]:
model = nw.Model(X_tr,y_tr,X_te,y_te,None,None)

In [ ]:
model.reset()
model.add("dense1",X_tr.shape[1],500,"relu")
model.add("dense2",500,250,"relu")
model.add("dense3",250,150,"relu")
model.add("dense4",150,100,"sine")
model.add("dense5",100,60,"sine")
model.add("dense6",60,30,"relu")
model.add("dense7",30,15,"tanh")
model.add("dense8",15,10,"softmax")

In [ ]:
model.summary()

In [ ]:
model.fit(X_tr,y_tr,0.0005,15,"Adam","Multi",mb=512,alpha_decay=False)

In [ ]:
model.test(X_te,y_te,"Multi")

In [ ]:
model.plot("Cost")

In [ ]:
model.plot("Accuracy")

## Replace contents of argument "direc" with the directory you want to save the images!

## For saving and loading the model, pass the directory in which you want to save the file as the argument and append it by the name of the file with .h5 extension!

In [ ]:
model.plot("Cost",animate=True,direc='/content/drive/My Drive/neowise/mnist/mnist-costs/')

In [ ]:
model.plot("Accuracy",animate=True,direc='/content/drive/My Drive/neowise/mnist/mnist-accu/')

In [ ]:
model.save_model('/content/drive/My Drive/neowise/mnist/mnist.h5')

In [ ]:
model1 = nw.Model(X_tr,y_tr,X_te,y_te,None,None)

In [ ]:
model1.load_model('/content/drive/My Drive/neowise/mnist/mnist.h5')

In [ ]:
model1.summary()

In [ ]:
model1.test(X_te,y_te,"Multi")

In [ ]:
act_te = model1.forward_prop(X_te,False)
predictions_te = nw.functional.PredictMulti(act_te["A" + str(len(model1.layer_names))])()
ones_arr = np.ones(predictions_te.shape)
temp1 = predictions_te == ones_arr
ind = []
for gee in range(0, temp1.shape[1]):
    for jee in range(0, temp1.shape[0]):
        if temp1[jee, gee] == True:
            ind.append(jee)
ind_arr = np.array(ind)

In [ ]:
perm = np.random.permutation(9999)
l = 0

fig, axs = plt.subplots(5, 5, figsize=(100,100))

for g in range(0,5):
  for h in range(0,5):
    foo = perm[l]
    second_image = X_test_orig[foo, :]
    second_label = Y_te_resh[foo]
    second_pred_label = ind_arr[foo]
    axs[h,g].set_title('Digit Label: {} Predicted Label: {}'.format(second_label,second_pred_label),fontsize=50)
    axs[h,g].imshow(second_image,cmap='gray_r')
    l = l+1